# Gerador de instâncias sintéticas para o problema de otimização do mestrado

### Etapas

**Geral**

1. Definir o grid

1. Definir bacias geológicas

1. Definir plays

1. Definir quantidade de projetos

**Para cada projeto:**

1. Escolher uma bacia

1. Sortear coordenada (x,y)

1. Pela bacia, temos a estratégia e a qualidade do dado

1. Dada a bacia, escolher um play

1. O play vai indicar o range de soterramento, e distribuição dos parâmetros volumétricos

1. O risco tem componentes regionais da bacia (geração), componentes do play (migração, retenção), e componentes individuais (geometria e reservatório).

1. A PCGNA é indicada pelo play

1. Simular volume geo, tec, ambos condicionais e incondicionais

1. Cada bacia tem uma $f(vol) \rightarrow VPL$

1. Calcular distribuição de VPL e métricas de retorno (risco?)

1. O custo do poço exploratório depende da bacia, estratégia e soterramento

1. Tempo de excecução dos projetos é função do soterramento e estratégia

1. Atribuir contratos à cada parte da bacia com OEs: simular leilões?

# Setup da instância

In [ ]:
n_plays = 10
n_projetos = 190
n_sondas = 10
iteracoes = 1000
t_init = 0
t_final = 5*12*4*7
delta_t = 1*7*4

In [ ]:
estrategias = [1, 2, 3]
qualidades_dado = [1, 2, 3]

## Importando bibliotecas

In [ ]:
import math
import copy
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import cm

%matplotlib inline

## Definição do grid

In [ ]:
x_min, x_max = (0., 100.)
y_min, y_max = (0., 100.)
dx, dy = (1., 1.)
grid = np.zeros(( int((y_max-y_min)/dy), int((x_max-x_min)/dx) ))

In [ ]:
grid.shape

In [ ]:
plt.pcolormesh(grid, alpha=0.5)

## Criando bacias geológicas

In [ ]:
nx_bacias = 3
ny_bacias = 3
bacias = {}
count = 0
x_step = (x_max-x_min)/nx_bacias
y_step = (y_max-y_min)/ny_bacias
for i in range(1, ny_bacias+1):
    y_min_curr = (i-1)*y_step
    y_max_curr = (i)*y_step
    for j in range(1, nx_bacias+1):
        x_min_curr = (j-1)*x_step
        x_max_curr = j*x_step
        count += 1
        bacias[count] = {}
        bacias[count]['Coordenadas'] = ((x_min_curr, x_max_curr),(y_min_curr, y_max_curr))
        bacias[count]['Maturidade'] = np.random.choice(estrategias)
        bacias[count]['Qualidade do dado'] = np.random.choice(qualidades_dado)
        grid[int(x_min_curr):int(x_max_curr), int(y_min_curr):int(y_max_curr)] = count

In [ ]:
bacias

In [ ]:
colormap_dict = cm.cmap_d.keys()
colormap_dict

In [ ]:
colormap = cm.inferno
plt.figure(figsize=(7, 7))
plt.pcolormesh(grid, alpha=0.5, cmap=colormap)

## Criando projetos

In [ ]:
plays = [i for i in range(1, n_plays+1)]

In [ ]:
projetos = {}
projetos['Coordenada x'] = []
projetos['Coordenada y'] = []
projetos['Bacia'] = []
projetos['Nome'] = []
projetos['Maturidade bacia'] = []
projetos['Qualidade do dado'] = []
projetos['Play'] = []

for i in range(n_projetos):
    
    # primeiro, sortear uma bacia
    bacia = np.random.choice(list(bacias.keys()))
    
    # não permitir coordenadas nas bordas das bacias
    fator = 0.8
    x_min_curr, x_max_curr = bacias[bacia]['Coordenadas'][0]
    y_min_curr, y_max_curr = bacias[bacia]['Coordenadas'][1]
    
    x_proj, y_proj = (np.random.uniform(x_min_curr + x_step*(1-fator), x_max_curr - x_step*(1-fator)), 
                      np.random.uniform(y_min_curr + y_step*(1-fator), y_max_curr - y_step*(1-fator)))
    
    projetos['Nome'].append(i+1)
    projetos['Bacia'].append(bacia)
    projetos['Coordenada x'].append(x_proj)
    projetos['Coordenada y'].append(y_proj)
    projetos['Maturidade bacia'].append(bacias[bacia]['Maturidade'])
    projetos['Qualidade do dado'].append(bacias[bacia]['Qualidade do dado'])
    projetos['Play'].append(np.random.choice(plays))

In [ ]:
plt.scatter(projetos['Coordenada x'], projetos['Coordenada y'])

In [ ]:
fig = plt.figure(figsize=(7, 7), frameon=False)

im1 = plt.pcolormesh(grid, alpha=0.5, cmap=colormap)
im2 = plt.scatter(projetos['Coordenada x'], projetos['Coordenada y'], color=(0.1, 0.8, 0.8), s=50, marker='*')

plt.show()

In [ ]:
proj_df = pd.DataFrame(projetos)

In [ ]:
proj_df

In [ ]:
proj_df['Bacia'].unique(), proj_df['Maturidade bacia'].unique(), proj_df['Qualidade do dado'].unique(), proj_df['Play'].unique()

## Definindo parâmetros volumétricos e risco geológico

In [ ]:
soterramento_range = (1000., 5000.)

pcgna_range = (0., 1.)

area_range = (20., 200.)
netpay_range = (5., 100.)
phi_range = (0.10, 0.32)
trapfill_range = (0.5, 1.)
shc_range = (0.5, 1.)
bo_range = (1.4, 1.9)
bg_range = (0.002, 0.004)
rgo_range = (150, 250)
rlg_range = (0.1, 0.15)
froleo_range = (0.15, 0.25)
frga_range = (0.15, 0.25)
frgna_range = (0.15, 0.25)
frcond_range = (0.15, 0.25)

### TODO: colocar correlação nas variáveis aleatórias

In [ ]:
plays_completo = {}
for p in plays:
    plays_completo[copy.copy(p)] = {}
    
    loc, scale = soterramento_range[0], soterramento_range[1]-soterramento_range[0]
    plays_completo[copy.copy(p)]['Soterramento'] = stats.uniform(loc, scale).rvs(2)
    
    loc, scale = pcgna_range[0], pcgna_range[1] - pcgna_range[0]
    plays_completo[copy.copy(p)]['pcgna'] = stats.uniform(loc, scale).rvs(2)
    
    loc, scale = area_range[0], area_range[1] - area_range[0]
    plays_completo[copy.copy(p)]['area'] = stats.uniform(loc, scale).rvs(2)
    
    loc, scale = netpay_range[0], netpay_range[1] - netpay_range[0]
    plays_completo[copy.copy(p)]['netpay'] = stats.uniform(loc, scale).rvs(2)
    
    loc, scale = phi_range[0], phi_range[1] - phi_range[0]
    plays_completo[copy.copy(p)]['phi'] = stats.uniform(loc, scale).rvs(2)
    
    loc, scale = trapfill_range[0], trapfill_range[1] - trapfill_range[0]
    plays_completo[copy.copy(p)]['trapfill'] = stats.uniform(loc, scale).rvs(2)
    
    loc, scale = shc_range[0], shc_range[1] - shc_range[0]
    plays_completo[copy.copy(p)]['shc'] = stats.uniform(loc, scale).rvs(2)
    
    loc, scale = bo_range[0], bo_range[1] - bo_range[0]
    plays_completo[copy.copy(p)]['bo'] = stats.uniform(loc, scale).rvs(2)
    
    loc, scale = bg_range[0], bg_range[1] - bg_range[0]
    plays_completo[copy.copy(p)]['bg'] = stats.uniform(loc, scale).rvs(2)
    
    loc, scale = rgo_range[0], rgo_range[1] - rgo_range[0]
    plays_completo[copy.copy(p)]['rgo'] = stats.uniform(loc, scale).rvs(2)
    
    loc, scale = rlg_range[0], rlg_range[1] - rlg_range[0]
    plays_completo[copy.copy(p)]['rlg'] = stats.uniform(loc, scale).rvs(2)
    
    loc, scale = froleo_range[0], froleo_range[1] - froleo_range[0]
    plays_completo[copy.copy(p)]['froleo'] = stats.uniform(loc, scale).rvs(2)
    
    loc, scale = frga_range[0], frga_range[1] - frga_range[0]
    plays_completo[copy.copy(p)]['frga'] = stats.uniform(loc, scale).rvs(2)
    
    loc, scale = frgna_range[0], frgna_range[1] - frgna_range[0]
    plays_completo[copy.copy(p)]['frgna'] = stats.uniform(loc, scale).rvs(2)
    
    loc, scale = frcond_range[0], frcond_range[1] - frcond_range[0]
    plays_completo[copy.copy(p)]['frcond'] = stats.uniform(loc, scale).rvs(2)

In [ ]:
plays_completo

### TODO: escalar incerteza pela qualidade do dado e maturidade da bacia

### TODO: colocar fatores de risco com compentes por bacia, play, e prospecto

In [ ]:
def ComputeDist(p90_play, p10_play):
    loc, scale = p90_play, p10_play - p90_play
    p90_proj, p10_proj = sorted(stats.uniform(loc, scale).rvs(2))
    mu = (np.log(p90_proj) + np.log(p10_proj)) / 2
    sigma = (np.log(p10_proj)-mu) / 1.2815515655446
    dist = stats.lognorm(s=sigma, scale=np.exp(mu))
    return dist

projetos['Soterramento'] = []
projetos['pcgna'] = []
projetos['area'] = []
projetos['netpay'] = []
projetos['phi'] = []
projetos['trapfill'] = []
projetos['shc'] = []
projetos['bo'] = []
projetos['bg'] = []
projetos['rgo'] = []
projetos['rlg'] = []
projetos['froleo'] = []
projetos['frga'] = []
projetos['frgna'] = []
projetos['frcond'] = []
projetos['geração'] = []
projetos['migração'] = []
projetos['reservatório'] = []
projetos['geometria'] = []
projetos['retenção'] = []
projetos['PS HC'] = []

for k, p in enumerate(projetos['Nome']):
    
    play = projetos['Play'][k]
    
    soterr_p90_play, soterr_p10_play = sorted(plays_completo[play]['Soterramento']) # soterramento
    pcgna_p90_play, pcgna_p10_play = sorted(plays_completo[play]['pcgna']) # pcgna 
    area_p90_play, area_p10_play = sorted(plays_completo[play]['area']) # area
    netpay_p90_play, netpay_p10_play = sorted(plays_completo[play]['netpay']) # netpay 
    phi_p90_play, phi_p10_play = sorted(plays_completo[play]['phi']) # phi 
    trapfill_p90_play, trapfill_p10_play = sorted(plays_completo[play]['trapfill']) # trapfill 
    shc_p90_play, shc_p10_play = sorted(plays_completo[play]['shc']) # shc 
    bo_p90_play, bo_p10_play = sorted(plays_completo[play]['bo']) # bo 
    bg_p90_play, bg_p10_play = sorted(plays_completo[play]['bg']) # bg 
    rgo_p90_play, rgo_p10_play = sorted(plays_completo[play]['rgo']) # rgo 
    rlg_p90_play, rlg_p10_play = sorted(plays_completo[play]['rlg']) # rlg 
    froleo_p90_play, froleo_p10_play = sorted(plays_completo[play]['froleo']) # froleo 
    frga_p90_play, frga_p10_play = sorted(plays_completo[play]['frga']) # frga 
    frgna_p90_play, frgna_p10_play = sorted(plays_completo[play]['frgna']) # frgna
    frcond_p90_play, frcond_p10_play = sorted(plays_completo[play]['frcond']) # frcond
    
    loc, scale = soterr_p90_play, soterr_p10_play - soterr_p90_play
    soterramento = stats.uniform(loc, scale).rvs(1)[0]
    projetos['Soterramento'].append(soterramento)
    
    loc, scale = pcgna_p90_play, pcgna_p10_play - pcgna_p90_play
    pcgna = stats.uniform(loc, scale).rvs(1)[0]
    projetos['pcgna'].append(pcgna)
    
    # area
    area_dist = ComputeDist(area_p90_play, area_p10_play)
    projetos['area'].append(area_dist)
    # netpay
    netpay_dist = ComputeDist(netpay_p90_play, netpay_p10_play)
    projetos['netpay'].append(netpay_dist)
    # phi
    phi_dist = ComputeDist(phi_p90_play, phi_p10_play)
    projetos['phi'].append(phi_dist)
    # trapfill
    trapfill_dist = ComputeDist(trapfill_p90_play, trapfill_p10_play)
    projetos['trapfill'].append(trapfill_dist)
    # shc
    shc_dist = ComputeDist(shc_p90_play, shc_p10_play)
    projetos['shc'].append(shc_dist)
    # bo
    bo_dist = ComputeDist(bo_p90_play, bo_p10_play)
    projetos['bo'].append(bo_dist)
    # bg
    bg_dist = ComputeDist(bg_p90_play, bg_p10_play)
    projetos['bg'].append(bg_dist)
    # rgo
    rgo_dist = ComputeDist(rgo_p90_play, rgo_p10_play)
    projetos['rgo'].append(rgo_dist)
    # rlg
    rlg_dist = ComputeDist(rlg_p90_play, rlg_p10_play)
    projetos['rlg'].append(rlg_dist)
    # froleo
    froleo_dist = ComputeDist(froleo_p90_play, froleo_p10_play)
    projetos['froleo'].append(froleo_dist)
    # frga
    frga_dist = ComputeDist(frga_p90_play, frga_p10_play)
    projetos['frga'].append(frga_dist)
    # frgna
    frgna_dist = ComputeDist(frgna_p90_play, frgna_p10_play)
    projetos['frgna'].append(frgna_dist)
    # frcond
    frcond_dist = ComputeDist(frcond_p90_play, frcond_p10_play)
    projetos['frcond'].append(frcond_dist)
    
    geracao = stats.uniform(0.5, 0.6).rvs(1)[0]
    projetos['geração'].append(geracao)
    migracao = stats.uniform(0.5, 0.6).rvs(1)[0]
    projetos['migração'].append(migracao)
    reservatorio = stats.uniform(0.5, 0.6).rvs(1)[0]
    projetos['reservatório'].append(reservatorio)
    geometria = stats.uniform(0.5, 0.6).rvs(1)[0]
    projetos['geometria'].append(geometria)
    retencao = stats.uniform(0.5, 0.6).rvs(1)[0]
    projetos['retenção'].append(retencao)
    projetos['PS HC'].append( geracao*migracao*reservatorio*geometria*retencao )
    

In [ ]:
proj_df = pd.DataFrame(projetos)

In [ ]:
proj_df.head()

In [ ]:
proj_df[['geração', 'migração', 'reservatório', 'geometria', 'retenção', 'PS HC']].describe()

In [ ]:
list(projetos.keys())

## Simulação dos volumes condicionais e incondicionais (Monte Carlo)

In [ ]:
projetos['volume oleo'] = []
projetos['volume ga'] = []
projetos['volume gna'] = []
projetos['volume cond'] = []
projetos['volume condicional'] = []
projetos['média condicional'] = []
projetos['volume incondicional'] = []
projetos['média incondicional'] = []

for k in range(n_projetos):
    
    volume_oleo_dist = []
    volume_ga_dist = []
    volume_gna_dist = []
    volume_cond_dist = []
    volume_hc_dist = []
    volume_condicional_dist = []
    volume_incondicional_dist = []
    
    for i in range(iteracoes):
        
        # sortear se é uma descoberta
        descoberta = stats.uniform(0, 1).rvs(1)[0] < projetos['PS HC'][k]
        
        # sortear o flúido principal
        fluido = 'óleo' if stats.uniform(0, 1).rvs(1)[0] < projetos['pcgna'][k] else 'gás'
        
        # dado o flúido principal, calcular volumes
        area = projetos['area'][k].rvs(1)[0]
        netpay = projetos['netpay'][k].rvs(1)[0]
        phi = projetos['phi'][k].rvs(1)[0]
        trapfill = projetos['trapfill'][k].rvs(1)[0]
        shc = projetos['shc'][k].rvs(1)[0]
        bo = projetos['bo'][k].rvs(1)[0]
        bg = projetos['bg'][k].rvs(1)[0]
        rgo = projetos['rgo'][k].rvs(1)[0]
        rlg = projetos['rlg'][k].rvs(1)[0]
        froleo = projetos['froleo'][k].rvs(1)[0]
        frga = projetos['frga'][k].rvs(1)[0]
        frgna = projetos['frgna'][k].rvs(1)[0]
        frcond = projetos['frcond'][k].rvs(1)[0]
        
        if fluido == 'óleo':
            comum = area * netpay * phi * trapfill * shc
            volume_oleo = ((comum * froleo) / bo) / (0.159)
            volume_ga = ((comum * rgo * frga) / bo)/(1000*0.159)
            volume_hc = volume_oleo + volume_ga
            volume_oleo_dist.append(volume_oleo)
            volume_ga_dist.append(volume_ga)
        elif fluido == 'gás':
            comum = area * netpay * phi * trapfill * shc
            volume_gna = ((comum * frgna) / bg) / (1000*0.159)
            volume_cond = ((comum * rlg * frcond) / bg) / (1000*0.159)
            volume_hc = volume_gna + volume_cond
            volume_gna_dist.append(volume_gna)
            volume_cond_dist.append(volume_cond)
        
        volume_hc_incond = volume_hc if descoberta else 0
        volume_condicional_dist.append(volume_hc)
        volume_incondicional_dist.append(volume_hc_incond)
        
    projetos['volume oleo'].append(volume_oleo_dist)
    projetos['volume ga'].append(volume_ga_dist)
    projetos['volume gna'].append(volume_gna_dist)
    projetos['volume cond'].append(volume_cond_dist)
    projetos['volume condicional'].append(volume_condicional_dist)
    projetos['média condicional'].append(np.mean(volume_condicional_dist))
    projetos['volume incondicional'].append(volume_incondicional_dist)
    projetos['média incondicional'].append(np.mean(volume_incondicional_dist))

In [ ]:
proj_df = pd.DataFrame(projetos)

In [ ]:
proj_df.head()

In [ ]:
def VisualizarVA(dist):
    
    p10, p90 = np.percentile(dist, 10), np.percentile(dist, 90)
    
    plt.hist(dist, bins=20, cumulative=False, density=True, align='right')
    plt.axvline(p10, color='r')
    plt.axvline(p90, color='r')
    plt.xlabel('classes')
    plt.ylabel('probabilidade acumulada');

In [ ]:
for i in range(n_projetos):
    plt.figure(i)
    VisualizarVA(projetos['volume condicional'][i])

In [ ]:
for i in range(n_projetos):
    plt.figure(i)
    VisualizarVA(projetos['volume incondicional'][i])

## Calibração econômica: $f(volume) \rightarrow VPL$

### TODO: variar calibração por maturidade da bacia

In [ ]:
def AvalEcon(vol, fixo, unit):
    vpl = - fixo + (vol * unit)
    return vpl

def CalibEcon(dist, fixo, unit):
    vpl_dist = []
    for v in dist:
        vpl = AvalEcon(v, fixo, unit)
        vpl_dist.append(vpl)
    return vpl_dist

In [ ]:
projetos['VPL condicional'] = []
projetos['VPL incondicional'] = []
projetos['VPL condicional médio'] = []
projetos['VPL incondicional médio'] = []
for i in range(n_projetos):
    vpl_dist_cond = CalibEcon(projetos['volume condicional'][i], fixo=500, unit=5)
    vpl_dist_incond = CalibEcon(projetos['volume incondicional'][i], fixo=500, unit=5)
    projetos['VPL condicional'].append(copy.copy(vpl_dist_cond))
    projetos['VPL incondicional'].append(copy.copy(vpl_dist_incond))
    projetos['VPL condicional médio'].append(np.mean(vpl_dist_cond))
    projetos['VPL incondicional médio'].append(np.mean(vpl_dist_incond))

In [ ]:
for i in range(n_projetos):
    plt.figure(i)
    VisualizarVA(projetos['VPL condicional'][i])

In [ ]:
proj_df = pd.DataFrame(projetos)

In [ ]:
proj_df.head()

## Calculando custo por poço perfurado

In [ ]:
def CustoPoco(soterramento, maturidade):
    unit_loc = {3: 2,
                2: 1.5,
                1: 1}
    unit = 20000
    custo = soterramento*unit*unit_loc[maturidade]
    return custo / 10**6
    

In [ ]:
projetos['Custo do poço'] = []
for i in range(n_projetos):
    projetos['Custo do poço'].append(CustoPoco(projetos['Soterramento'][i], projetos['Maturidade bacia'][i]))

In [ ]:
proj_df = pd.DataFrame(projetos)

In [ ]:
proj_df.head()

## Calculando tempo de execução por poço perfurado

In [ ]:
def TempoExecucao(soterramento, maturidade):
    unit_loc = {3: 3,
                2: 2,
                1: 1}
    unit = (100 / 5000)
    tempo = soterramento*unit*unit_loc[maturidade]
    return max(tempo, 28.)

In [ ]:
projetos['Tempo de execução'] = []
for i in range(n_projetos):
    projetos['Tempo de execução'].append(int(TempoExecucao(projetos['Soterramento'][i], projetos['Maturidade bacia'][i])))

In [ ]:
proj_df = pd.DataFrame(projetos)

In [ ]:
proj_df.head()

## Atribuindo contratos aos projetos

In [ ]:
duracao = [1, 2, 3, 4, 5]
projetos['Início do Contrato'] = []
projetos['Final do Contrato'] = []
for i in range(n_projetos):
    inicio = np.random.randint(0, (5 * 12 * 4 * 7))
    final = inicio + (12 * 4 * 7)*np.random.choice(duracao)
    final = min(final, (5 * 12 * 4 * 7))
    projetos['Início do Contrato'].append(int(copy.copy(inicio)))
    projetos['Final do Contrato'].append(int(copy.copy(final)))

In [ ]:
proj_df = pd.DataFrame(projetos)

In [ ]:
proj_df.head()

## Criando conjunto de sondas de perfuração

In [ ]:
sondas = {}
sondas['Coordenada x'] = []
sondas['Coordenada y'] = []

for i in range(n_sondas):
    
    # primeiro, sortear uma bacia
    bacia = np.random.choice(list(bacias.keys()))
    
    # colocar coordenadas nos extremos verticais da bacia
    x_min_curr, x_max_curr = bacias[bacia]['Coordenadas'][0]
    y_min_curr, y_max_curr = bacias[bacia]['Coordenadas'][1]
    
    x_rig, y_rig = (0.0, np.random.choice([y_min_curr, y_max_curr]))
    
    sondas['Coordenada x'].append(x_rig)
    sondas['Coordenada y'].append(y_rig)

In [ ]:
sondas

## Visualizando métricas dos projetos para controle de qualidade dos dados

In [ ]:
proj_df[['Soterramento', 'média incondicional', 'VPL incondicional médio', 'Custo do poço', 'Tempo de execução']].describe()

## Codificando variáveis categóricas para a exportação

### TODO: codificar categóricas

## Definindo períodos de tempo

In [ ]:
n_periodos = ((t_final - t_init) // delta_t)

## Convertendo dados de tempo para períodos

In [ ]:
for i in range(n_projetos):
    projetos['Tempo de execução'][i] = int(projetos['Tempo de execução'][i] // delta_t)
    projetos['Início do Contrato'][i] = int(projetos['Início do Contrato'][i] // delta_t)
    projetos['Final do Contrato'][i] = int(projetos['Final do Contrato'][i] // delta_t)

## Calcula tempos de deslocamento

In [ ]:
desloc = []
lag = n_sondas
for i in range(n_projetos+n_sondas):
    desloc.append([])
    for j in range(n_projetos+n_sondas):
        if (i<n_sondas) and (j<n_sondas):
            dist = math.sqrt( (sondas['Coordenada x'][i] - sondas['Coordenada x'][j])**2 + 
                             (sondas['Coordenada y'][i] - sondas['Coordenada y'][j])**2 )
        elif (i<n_sondas) and not(j<n_sondas):
            dist = math.sqrt( (sondas['Coordenada x'][i] - projetos['Coordenada x'][j-lag])**2 + 
                             (sondas['Coordenada y'][i] - projetos['Coordenada y'][j-lag])**2 )
        elif not(i<n_sondas) and (j<n_sondas):
            dist = math.sqrt( (projetos['Coordenada x'][i-lag] - sondas['Coordenada x'][j])**2 + 
                             (projetos['Coordenada y'][i-lag] - sondas['Coordenada y'][j])**2 )
        else:
            dist = math.sqrt( (projetos['Coordenada x'][i-lag] - projetos['Coordenada x'][j-lag])**2 + 
                             (projetos['Coordenada y'][i-lag] - projetos['Coordenada y'][j-lag])**2 )

        desloc[i].append(int(dist // delta_t))

## Exportando instância

In [ ]:
colunas_out_proj = ['Coordenada x', 'Coordenada y', 'Bacia', 'Nome', 'Maturidade bacia',
               'Qualidade do dado', 'Play', 'Soterramento', 'pcgna', 'geração', 'migração', 
               'reservatório', 'geometria', 'retenção', 'PS HC', 'média condicional', 'média incondicional', 
               'VPL condicional médio', 'VPL incondicional médio', 'Custo do poço', 
               'Tempo de execução', 'Início do Contrato', 'Final do Contrato']

In [ ]:
for col in colunas_out_proj:
    for i, it in enumerate(projetos[col]):
        projetos[col][i] = round(projetos[col][i], 2)

In [ ]:
colunas_out_sondas = ['Coordenada x', 'Coordenada y']

In [ ]:
for col in colunas_out_sondas:
    for i, it in enumerate(sondas[col]):
        sondas[col][i] = round(sondas[col][i], 2)

In [ ]:
with open('instancia_' + str(n_projetos) + 'projetos_' + 
          str(n_sondas) + 'sondas_delta_t' + str(delta_t) + '.dat', 'w') as f:
    print (n_projetos, sum(projetos['Custo do poço'])/np.random.choice([1, 2, 3, 4]), t_init, t_final, delta_t, n_periodos, file=f)
    for col in colunas_out_proj:
        print(projetos[col], file=f)
    print (n_sondas, file=f)
    print (sondas['Coordenada x'], file=f)
    print (sondas['Coordenada y'], file=f)
    print (desloc, file=f)